In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import ConnectionPatch
import warnings
import snowflake.connector
import sys
import os
import re
warnings.filterwarnings('ignore')

PN='P1000194588'
PN_string = PN,PN
my_query = f"""
select
ltrim(a.aufnr,'0') as production_order,
a.WERKS as plant,
b.GLTRI as actual_finish_date,
ltrim(b.plnbez,'0') as PN0,
h.description as description,
ltrim(c.matnr,'0') as PN1,
c.wewrt as accounting_cost,
ltrim(d.kstar,'0') as cost_element,
d.wrttp as cost_type,
d.WTG001,d.WTG002,d.WTG003,d.WTG004,d.WTG005,d.WTG006,d.WTG007,
d.WTG008,d.WTG009,d.WTG010,d.WTG011,d.WTG012,d.WTG013,d.WTG014,
d.WTG015,d.WTG016
from rpl_sap_attunity.aufk as a
inner join rpl_sap_attunity.afko as b on b.aufnr = a.aufnr
inner join core.material as h on h.material = b.plnbez
inner join rpl_sap_attunity.afpo as c on c.aufnr = a.aufnr
inner join rpl_sap_attunity.coss as d on d.objnr = a.objnr
where b.terkz ='1'
and c.elikz ='X'
and d.wrttp ='04'
--and a.WERKS = '2101'
and PN0 in {PN_string}
order by a.aufnr
"""    # internal cost query 

my_query1 = f"""
select 
ltrim(a.aufnr,'0') as production_order,
a.WERKS as plant,
b.GLTRI as actual_finish_date,
ltrim(b.plnbez,'0') as PN0,
ltrim(c.matnr,'0') as PN1,
c.wewrt as accounting_cost,
ltrim(d.kstar,'0') as cost_element,
d.wrttp as cost_type,
d.WTG001,d.WTG002,d.WTG003,d.WTG004,d.WTG005,d.WTG006,d.WTG007,d.WTG008,
d.WTG009,d.WTG010,d.WTG011,d.WTG012,d.WTG013,d.WTG014,d.WTG015,d.WTG016
from rpl_sap_attunity.aufk as a
inner join rpl_sap_attunity.afko as b on b.aufnr = a.aufnr
inner join rpl_sap_attunity.afpo as c on c.aufnr = a.aufnr
inner join rpl_sap_attunity.cosp as d on d.objnr = a.objnr
where b.terkz ='1'
and c.elikz ='X'
and d.wrttp ='04'
and PN0 in {PN_string}
--and a.WERKS = '2101'
order by a.aufnr
"""  # query statement

my_query2 = f"""
select 
ltrim(a.aufnr,'0') as production_order,
a.WERKS as plant,
b.GLTRI as actual_finish_date,
ltrim(b.plnbez,'0') as PN0,
d.matnr as component_pn,
f.description as description,
case when d.waers ='USD'
    then d.ENWRT 
    else round(d.enwrt * g.exchange_rate,2)
end as cot_total_cost_usd,
d.waers as orin_currency,
d.enmng as qty_consumption,
d.saknr as cost_element,
d.erfmg as qty_per_unit,
d.ERFME as UOM,
d.baugr as toplevel_pn,
case when d.waers ='USD'
    then d.GPREIS 
    else round(d.GPREIS * g.exchange_rate,2)
end as unit_cost_usd
from rpl_sap_attunity.aufk as a
inner join rpl_sap_attunity.afko as b on b.aufnr = a.aufnr
inner join rpl_sap_attunity.afpo as c on c.aufnr = a.aufnr
inner join (select matnr,ENWRT,waers,aufnr,enmng,saknr,erfmg,erfme,baugr,gpreis,sbter,concat(waers::text,substring(sbter::text,1,8)) as 
                      uni_key from rpl_sap_attunity.resb ) as d on d.aufnr = a.aufnr
inner join core.material as f on f.material = d.matnr
left join (select from_currency, to_currency, exchange_rate,rate_date,concat(from_currency::text,substring(to_char(rate_date,'yyyymmdd')::text,1,8)) as 
                                    uni_key from SAP_REPORTING.CURRENCY_CONVERSION where to_currency='USD') as g on g.uni_key = d.uni_key
where b.terkz ='1'
and c.elikz ='X'
and PN0 in {PN_string}
order by a.aufnr
""" # query the purchased individual part cost 


# query data from snowfake
with snowflake.connector.connect( 
    user='ethan.zhu@technipfmc.com', # Required. Replace with your email 
    authenticator="externalbrowser", # Required. 
    account='technipfmc-data', # Required. 
    database="idsprod", # Optional 
    schema="rpl_sap.ekko", # Optional. Replace with the schema you will be working on 
    role="reporting", # Optional. Replace with the role you will be working with 
    warehouse="reporting_wh", # Optional. Replace with the warehouse you will be working with 
    client_store_temporary_credential=True, # Only if installing secure-local-storage to avoid reopening tabs
    ) as conn: 
    cursor = conn.cursor()
    cursor.execute(my_query)
    cursor1 = conn.cursor()
    cursor1.execute(my_query1)
    cursor2 = conn.cursor()
    cursor2.execute(my_query2)
    # res = cursor.fetchall() # To return a list of tuples 
    df_toplevelpn = cursor.fetch_pandas_all() # To return a dataframe
    df_externalcost = cursor1.fetch_pandas_all()
    df_individualpn = cursor2.fetch_pandas_all()
print(df_toplevelpn.head(2),df_toplevelpn.shape)

  PRODUCTION_ORDER PLANT ACTUAL_FINISH_DATE          PN0  \
0        302316713  2101           20220322  P1000194588   
1        302316714  2101           20220324  P1000194588   

                                         DESCRIPTION          PN1  \
0  ACTUATOR ASSY F/ AH700 (SUBSEA 2.0) 2 1/16-15K...  P1000194588   
1  ACTUATOR ASSY F/ AH700 (SUBSEA 2.0) 2 1/16-15K...  P1000194588   

   ACCOUNTING_COST COST_ELEMENT COST_TYPE  WTG001  ...  WTG007  WTG008  \
0          9943.83       497142        04     0.0  ...     0.0     0.0   
1          9943.83       497142        04     0.0  ...     0.0     0.0   

   WTG009  WTG010  WTG011  WTG012  WTG013  WTG014  WTG015  WTG016  
0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0  
1     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0  

[2 rows x 25 columns] (14, 25)


In [2]:
os.getcwd()

'c:\\Users\\zhusj\\python\\Project'

In [3]:
print(os.getcwd())

c:\Users\zhusj\python\Project


In [4]:
print(os.listdir())

['.gitignore', '.ipynb_checkpoints', 'BOM_organized.py', 'BOM_organizedV1.py', 'BOM_organizedV2.py', 'BOM_organizedV3.py', 'BOM_organizedV4.py', 'BOM_organizedV5.py', 'BOM_organized_orin.py', 'ChinaSpendAnalysis.ipynb', 'cnn-with-pytorch-0-995-accuracy.ipynb', 'Crab_competition_zhu.py', 'Function test .ipynb', 'function_test_all.ipynb', 'Historical PO of GSP.py', 'NN.ipynb', 'OEMQuery.py', 'Practical transformer.ipynb', 'SFdataQuery.py', 'Spider_tetst.py', 'System_level_cost_analysis.py', 'System_level_cost_analysisV1.py', 'Test.ipynb', 'Test_cls_fun.py', 'Text classification with Transformers.ipynb', 'Text classification with Transformers.py', 'Tranformer_concise.py', 'Transformer.ipynb', 'Transformer2.ipynb', 'Transformer2.py', 'Transformer_video.py', 'Word2Vec-Skipgram(Softmax).py']


In [5]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir) 

In [6]:
print(parent_dir)

c:\Users\zhusj\python


In [7]:
os.path.join(parent_dir,'data/order.xlsx')

'c:\\Users\\zhusj\\python\\data/order.xlsx'

In [8]:
os.path.join(parent_dir,r'data/order.xlsx')

'c:\\Users\\zhusj\\python\\data/order.xlsx'

In [9]:
pd.read_excel(os.path.join(parent_dir,r'data/order.xlsx'))

,Unnamed: 0,PRODUCTION_ORDER,PLANT,ACTUAL_FINISH_DATE,PN0,DESCRIPTION,PN1,ACCOUNTING_COST,COST_ELEMENT,COST_TYPE,...,WTG007,WTG008,WTG009,WTG010,WTG011,WTG012,WTG013,WTG014,WTG015,WTG016
0,0,302316713,2101,20220322,P1000194588,ACTUATOR ASSY F/ AH700 (SUBSEA 2.0) 2 1/16-15K...,P1000194588,9943.83,497142,4,...,0.0,0,0,0.0,0,0,0,0,0,0
1,1,302316714,2101,20220324,P1000194588,ACTUATOR ASSY F/ AH700 (SUBSEA 2.0) 2 1/16-15K...,P1000194588,9943.83,497142,4,...,0.0,0,0,0.0,0,0,0,0,0,0
2,2,302316715,2101,20220325,P1000194588,ACTUATOR ASSY F/ AH700 (SUBSEA 2.0) 2 1/16-15K...,P1000194588,9943.83,497142,4,...,0.0,0,0,0.0,0,0,0,0,0,0
3,3,302316716,2101,20220329,P1000194588,ACTUATOR ASSY F/ AH700 (SUBSEA 2.0) 2 1/16-15K...,P1000194588,9943.83,497142,4,...,0.0,0,0,0.0,0,0,0,0,0,0
4,4,302322864,2101,20220406,P1000194588,ACTUATOR ASSY F/ AH700 (SUBSEA 2.0) 2 1/16-15K...,P1000194588,9943.83,497142,4,...,0.0,0,0,0.0,0,0,0,0,0,0
5,5,302322865,2101,20220406,P1000194588,ACTUATOR ASSY F/ AH700 (SUBSEA 2.0) 2 1/16-15K...,P1000194588,9943.83,497142,4,...,0.0,0,0,0.0,0,0,0,0,0,0
6,6,302336909,2101,20220623,P1000194588,ACTUATOR ASSY F/ AH700 (SUBSEA 2.0) 2 1/16-15K...,P1000194588,9943.83,497142,4,...,0.0,0,0,0.0,0,0,0,0,0,0
7,7,302336909,2101,20220623,P1000194588,ACTUATOR ASSY F/ AH700 (SUBSEA 2.0) 2 1/16-15K...,P1000194588,9943.83,497145,4,...,0.0,0,0,0.0,0,0,0,0,0,0
8,8,302336911,2101,20220622,P1000194588,ACTUATOR ASSY F/ AH700 (SUBSEA 2.0) 2 1/16-15K...,P1000194588,9943.83,497142,4,...,0.0,0,0,0.0,0,0,0,0,0,0
9,9,302336911,2101,20220622,P1000194588,ACTUATOR ASSY F/ AH700 (SUBSEA 2.0) 2 1/16-15K...,P1000194588,9943.83,497145,4,...,0.0,0,0,0.0,0,0,0,0,0,0


In [10]:
os.path.join(parent_dir,f'data/{PN}order.xlsx')

'c:\\Users\\zhusj\\python\\data/P1000194588order.xlsx'

In [11]:
df_toplevelpn.to_excel(os.path.join(parent_dir,f'data/{PN}order.xlsx'))

In [12]:
pro_order = df_toplevelpn['PRODUCTION_ORDER'].unique()

In [13]:
type(pro_order)

numpy.ndarray

In [20]:
po_tup = [(i,j) for i,j in enumerate(pro_order)]

In [21]:
print(po_tup)

[(0, '302316713'), (1, '302316714'), (2, '302316715'), (3, '302316716'), (4, '302322864'), (5, '302322865'), (6, '302336909'), (7, '302336911'), (8, '302349369'), (9, '302375234'), (10, '302375235'), (11, '302377357')]


In [23]:
po_tup[0][1]

'302316713'

In [24]:
PN_info = {'PN':df_toplevelpn['PN0'][0],'Name':df_toplevelpn['DESCRIPTION'][0][:35],'PO_Number':len(po_tup),'PO':po_tup}

In [25]:
PN_info['PO_Number']

12

In [30]:
PN_info['Name']

'ACTUATOR ASSY F/ AH700 (SUBSEA 2.0)'